In [60]:
from abc import abstractmethod, ABC 
import pandas as pd 
from typing import Iterable, List, Optional
from datetime import datetime
import numpy as np

df = pd.read_csv("bloomberg_data_cleaned.csv")

#set date to datetime object 
df["date"] = pd.to_datetime(df["date"])
df.set_index(["date","ticker"],inplace= True)
pd.set_option('display.float_format', '{:,.4f}'.format)
df.head()

market_cap          volume  price_to_book  \
date       ticker                                                 
2022-01-03 A         47,257.0847    554,098.0000         8.7692   
           AA        11,293.5460  1,033,749.0000         2.3785   
           AAL       12,140.8973  8,786,970.0000            NaN   
           AAON       4,106.6876     83,790.0000         8.8262   
           AAPL   2,986,128.3180 20,242,137.0000        41.3474   

                   price_to_sales  pe_ratio  current_cap_share_class  \
date       ticker                                                      
2022-01-03 A               7.5274   42.0279              47,257.0847   
           AA              0.9249    8.9225              11,293.5460   
           AAL             0.4051       NaN              12,140.8973   
           AAON            7.6801   63.5297               4,106.6876   
           AAPL            7.9614   30.1565           2,986,128.3180   

                   close_price  
date       ticker               
2022-01-03 A          156.4800  
           AA          60.3600  
           AAL         18.7500  
           AAON        52.2200  
           AAPL       182.0100

In [38]:
''' 
========================================================
        Class for Updating and Improving our Dataframe
========================================================
Class Functionality: 
    (1): Add growth on a rolling basis: 
    (2): 

:::: Rolling Basis Methodology Explained::::

unfortunately, we are unable to calculated a 5 year rolling basis 
because we do not have five years of lookback data (we are going to
work on expanding are data include this). Instead we propose the 
below lookback logic as an "best-effort alternative. 

(1) 1st approach that's complicated and not consistent.  
start the benchmark price window on a date such that there is half a year of lookback
data. approximately 151 trading days. The lookback window will continue to expand 
until that rolling window is 2 years of 504 trading days approximately. At that

(2) 2nd approach. use a year lookback window. 

'''

def rolling_growth_rate(price_to_sales_series : pd.Series) -> float:  
    lookback_days = 252 #one trading year 
    """
    Returns: Na if No 
    """
    #see if there are NA values:
    NA_mask = price_to_sales_series.isna()
    if price_to_sales_series.loc[NA_mask].__len__() > 0:
        return np.nan
    print("being read")
    # else slope of linear fit is growth rate
    x_arbitrary = range(price_to_sales_series.__len__())
    slope, intercept = np.polyfit(x_arbitrary, price_to_sales_series,1) 
    return slope 


#df["rolling_test"] = df["price_to_sales"].rolling(252,min_periods=252).apply(rolling_growth_rate)
#df["rolling_test"].loc[df["rolling_test"].isna()]

df2 = DataframeHelper(df)





date        ticker
2022-01-03  A        NaN
            AA       NaN
            AAL      NaN
            AAON     NaN
            AAPL     NaN
                      ..
2025-12-31  ZG       NaN
            ZION     NaN
            ZM       NaN
            ZS       NaN
            ZTS      NaN
Name: rolling_test, Length: 1013030, dtype: float64

In [7]:
AAL_mask = df.index.get_level_values("ticker") == "AAL"
df.loc[AAL_mask]

,,market_cap,volume,price_to_book,price_to_sales,pe_ratio,current_cap_share_class,close_price
date,ticker,,,,,,,
2022-01-03,AAL,"12,140.8973","8,786,970.0000",NaN,0.4051,NaN,"12,140.8973",18.7500
2022-01-04,AAL,"12,315.7262","6,069,226.0000",NaN,0.4109,NaN,"12,315.7262",19.0200
2022-01-05,AAL,"12,095.5713","6,218,165.0000",NaN,0.4036,NaN,"12,095.5713",18.6800
2022-01-06,AAL,"12,024.3447","3,892,737.0000",NaN,0.4012,NaN,"12,024.3447",18.5700
2022-01-07,AAL,"12,484.0800","7,905,925.0000",NaN,0.4166,NaN,"12,484.0800",19.2800
...,...,...,...,...,...,...,...,...
2025-12-24,AAL,"10,350.1562","2,945,638.0000",NaN,0.1904,14.9055,"10,350.1562",15.6800
2025-12-26,AAL,"10,191.7355","3,738,749.0000",NaN,0.1875,14.6774,"10,191.7355",15.4400
2025-12-29,AAL,"9,993.7095","4,809,821.0000",NaN,0.1838,14.3922,"9,993.7095",15.1400


In [ ]:
class DataframeHelper:
    DATAOG: pd.DataFrame
    def __init__(self, data: pd.DataFrame) -> None:
        DataframeHelper.DATAOG = data 
        self.data = data 
    
    def slice_by_ticker(self,ticker: str) -> pd.DataFrame:
        ticker_mask = self.data.index.get_level_values("ticker") == ticker
        return self.data.loc[ticker_mask]

    def slice_by_day(self, day: datetime) -> pd.DataFrame: 
        day_mask = self.data.index.get_level_values("date") == day
        return self.data.loc[day_mask]
    
    def slice_any_row_with_na(self) -> pd.DataFrame:
        NA_mask = self.data.isna() 
        return self.data[NA_mask.any(axis=1)]
    
    def slice_complete_na_rows(self) -> pd.DataFrame: 
        #todo: 
        return pd.DataFrame()
    def slice_by_day_range(self, start_date: datetime, end_date: datetime) -> pd.DataFrame: 
        #todo 
        return pd.DataFrame() 
    
    @staticmethod
    def _rolling_growth_rate(price_to_sales_series : pd.Series) -> float:  
        lookback_days = 252 #one trading year 
        #see if there are NA values, there are growth is NA 
        NA_mask = price_to_sales_series.isna()
        if price_to_sales_series.loc[NA_mask].__len__() > 0:
            return np.nan
        # else slope of linear fit is growth rate
        x_arbitrary = range(price_to_sales_series.__len__())
        slope, intercept = np.polyfit(x_arbitrary, price_to_sales_series,1) 
        return slope*1000
        
    def add_roling_sales_growth_col(self):
        # add rolling sales growth 
        self.data["1year_PtoS_growth"] = 0.0
        for ticker in self.data.index.get_level_values("ticker").unique():
            df_slice = self.slice_by_ticker(ticker)
            df_slice["rolling_growth"] = df_slice["price_to_sales"].rolling(252,min_periods= 252).apply(self._rolling_growth_rate)
            icker_mask = df.index.get_level_values("ticker") == ticker
            self.data.loc[ticker,"1year_PtoS_growth"] = df_slice["rolling_growth"]

        
        


        



In [27]:
"""===============================================
        How to Use DataHelper Example 
==============================================="""
 
"""(1): I want just data on ticker AAL """
dhelp = DataframeHelper(df) 
dhelp.slice_by_ticker("AAL")

"""(2): Get data just from 2022,1,5 """
dhelp = DataframeHelper(df) 
dhelp.slice_by_day(datetime(2022,1,5))

"""(3): all rows with atleast one NA value """
dhelp.slice_any_row_with_na() 



market_cap         volume  price_to_book  price_to_sales  \
date       ticker                                                             
2022-01-03 AAL    12,140.8973 8,786,970.0000            NaN          0.4051   
           ADT     7,574.3542   430,701.0000         2.1948          1.3097   
           AFRM   26,754.5374 1,807,912.0000        10.9242         22.4449   
           ALAB           NaN            NaN            NaN             NaN   
           ALK     6,775.5478   590,412.0000         1.7910          1.0961   
...                       ...            ...            ...             ...   
2025-12-31 WYNN   12,511.2491   351,075.0000            NaN          1.7768   
           YUM    42,003.3200   448,630.0000            NaN          5.2416   
           Z      16,497.1620 1,002,015.0000            NaN             NaN   
           ZG     16,497.1620   495,003.0000         3.3081          6.6120   
           ZS     35,868.2641   341,534.0000        18.0936         12.3867   

                   pe_ratio  current_cap_share_class  close_price  
date       ticker                                                  
2022-01-03 AAL          NaN              12,140.8973      18.7500  
           ADT          NaN               7,113.4053       8.4200  
           AFRM         NaN              19,822.2759      95.2100  
           ALAB         NaN                      NaN          NaN  
           ALK          NaN               6,775.5478      54.0700  
...                     ...                      ...          ...  
2025-12-31 WYNN     21.5143              12,511.2491     120.3300  
           YUM      26.1948              42,003.3200     151.2800  
           Z            NaN              12,686.5149      68.2200  
           ZG           NaN               3,386.4618      68.2300  
           ZS           NaN              35,868.2641     224.9200  

[196825 rows x 7 columns]

In [20]:
df.to_parquet("clean_to_parquet_test.parquet")

In [25]:
df1 = pd.read_parquet("clean_to_parquet_test.parquet")
print(df1.dtypes, df1.index.dtype) 
df1

market_cap                 float64
volume                     float64
price_to_book              float64
price_to_sales             float64
pe_ratio                   float64
current_cap_share_class    float64
close_price                float64
dtype: object object


market_cap          volume  price_to_book  \
date       ticker                                                 
2022-01-03 A         47,257.0847    554,098.0000         8.7692   
           AA        11,293.5460  1,033,749.0000         2.3785   
           AAL       12,140.8973  8,786,970.0000            NaN   
           AAON       4,106.6876     83,790.0000         8.8262   
           AAPL   2,986,128.3180 20,242,137.0000        41.3474   
...                          ...             ...            ...   
2025-12-31 ZG        16,497.1620    495,003.0000         3.3081   
           ZION       8,642.9061    519,462.0000         1.2712   
           ZM        26,563.2794    658,825.0000         2.7598   
           ZS        35,868.2641    341,534.0000        18.0936   
           ZTS       55,448.0202    816,362.0000        10.2899   

                   price_to_sales  pe_ratio  current_cap_share_class  \
date       ticker                                                      
2022-01-03 A               7.5274   42.0279              47,257.0847   
           AA              0.9249    8.9225              11,293.5460   
           AAL             0.4051       NaN              12,140.8973   
           AAON            7.6801   63.5297               4,106.6876   
           AAPL            7.9614   30.1565           2,986,128.3180   
...                           ...       ...                      ...   
2025-12-31 ZG              6.6120       NaN               3,386.4618   
           ZION            1.7411   10.2570               8,642.9061   
           ZM              5.4384   23.2168              22,916.0108   
           ZS             12.3867       NaN              35,868.2641   
           ZTS             5.9776   20.2713              55,448.0202   

                   close_price  
date       ticker               
2022-01-03 A          156.4800  
           AA          60.3600  
           AAL         18.7500  
           AAON        52.2200  
           AAPL       182.0100  
...                        ...  
2025-12-31 ZG          68.2300  
           ZION        58.5400  
           ZM          86.2900  
           ZS         224.9200  
           ZTS        125.8200  

[1013030 rows x 7 columns]

In [36]:
dhelp = DataframeHelper(df) 
s = dhelp.slice_by_ticker("AAL")["price_to_sales"]
mask = s.isna() 
s[mask].__len__() 

0

In [69]:

def rolling_growth_rate(price_to_sales_series : pd.Series) -> float:  
    lookback_days = 252 #one trading year 
    """
    Returns: Na if No 
    """
    #see if there are NA values:
    NA_mask = price_to_sales_series.isna()
    if price_to_sales_series.loc[NA_mask].__len__() > 0:
        return np.nan
    # else slope of linear fit is growth rate
    x_arbitrary = range(price_to_sales_series.__len__())
    slope, intercept = np.polyfit(x_arbitrary, price_to_sales_series,1) 
    return slope*1000




dh = DataframeHelper(df)
df2 = dh.slice_by_ticker("XP") 
df2["rolling_test2"] = df2["price_to_sales"].rolling(252,min_periods=252).apply(rolling_growth_rate)
print(df2["rolling_test2"].iloc[500:620])

ticker_mask = df.index.get_level_values("ticker") == "XP"
df.loc[ticker_mask,"rolling_window"] = df2["rolling_test2"]
df.loc[ticker_mask]



date        ticker
2023-12-29  XP        8.3189
2024-01-02  XP        8.3416
2024-01-03  XP        8.3353
2024-01-04  XP        8.2929
2024-01-05  XP        8.2701
                       ...  
2024-06-14  XP       -4.4743
2024-06-17  XP       -4.5989
2024-06-18  XP       -4.7101
2024-06-20  XP       -4.7816
2024-06-21  XP       -4.8457
Name: rolling_test2, Length: 120, dtype: float64


C:\Users\lovaa_ovuf9q6\AppData\Local\Temp\ipykernel_42852\1025918298.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["rolling_test2"] = df2["price_to_sales"].rolling(252,min_periods=252).apply(rolling_growth_rate)


,,market_cap,volume,price_to_book,price_to_sales,pe_ratio,current_cap_share_class,close_price,rolling window,rolling_window
date,ticker,,,,,,,,,
2022-01-03,XP,"16,221.3175","593,088.0000",6.3717,7.2133,24.9346,"12,296.2168",28.9900,0,NaN
2022-01-04,XP,"15,740.1056","861,274.0000",6.2063,7.0260,24.2872,"11,931.4446",28.1300,0,NaN
2022-01-05,XP,"14,962.3329","1,781,434.0000",5.8933,6.6717,23.0623,"11,341.8709",26.7400,0,NaN
2022-01-06,XP,"15,158.1749","1,011,833.0000",5.9892,6.7803,23.4378,"11,490.3247",27.0900,0,NaN
2022-01-07,XP,"15,521.8816","1,107,360.0000",6.0625,6.8633,23.7246,"11,766.0246",27.7400,0,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-12-24,XP,"8,829.8668","674,270.0000",2.0317,2.7051,9.6522,"7,131.4036",16.4300,0,1.7623
2025-12-26,XP,"8,835.2410","941,425.0000",2.0338,2.7079,9.6621,"7,135.7441",16.4400,0,1.7129
2025-12-29,XP,"8,840.6153","1,071,219.0000",2.0500,2.7295,9.7392,"7,140.0846",16.4500,0,1.6652
